## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,21.20,79,90,11.50,light snow
1,1,Hermanus,ZA,-34.4187,19.2345,56.89,72,72,4.70,broken clouds
2,2,Burnie,AU,-41.0667,145.9167,60.49,48,8,10.45,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,74.97,78,100,13.85,moderate rain
4,4,Chuy,UY,-33.6971,-53.4616,53.76,83,10,2.77,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,74.97,78,100,13.85,moderate rain
9,9,Carnarvon,AU,-24.8667,113.6333,68.40,63,79,14.56,broken clouds
13,13,Karratha,AU,-20.7377,116.8463,79.77,40,1,15.10,clear sky
15,15,Acari,PE,-15.4311,-74.6158,68.22,73,59,3.62,broken clouds
18,18,Truth Or Consequences,US,33.1284,-107.2528,71.83,18,1,12.66,clear sky
19,19,Maceio,BR,-9.6658,-35.7353,80.04,94,75,6.91,broken clouds
20,20,Lagoa,PT,39.0500,-27.9833,68.16,86,3,23.80,clear sky
22,22,Saint-Joseph,RE,-21.3667,55.6167,71.96,80,18,8.75,few clouds
25,25,Nortelandia,BR,-14.4547,-56.8028,71.85,87,56,2.28,broken clouds
28,28,Kapaa,US,22.0752,-159.3190,79.48,61,40,4.00,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                303
City                   303
Country                301
Lat                    303
Lng                    303
Max Temp               303
Humidity               303
Cloudiness             303
Wind Speed             303
Current Description    303
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                301
City                   301
Country                301
Lat                    301
Lng                    301
Max Temp               301
Humidity               301
Cloudiness             301
Wind Speed             301
Current Description    301
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,74.97,moderate rain,-23.1203,-134.9692,
9,Carnarvon,AU,68.40,broken clouds,-24.8667,113.6333,
13,Karratha,AU,79.77,clear sky,-20.7377,116.8463,
15,Acari,PE,68.22,broken clouds,-15.4311,-74.6158,
18,Truth Or Consequences,US,71.83,clear sky,33.1284,-107.2528,
19,Maceio,BR,80.04,broken clouds,-9.6658,-35.7353,
20,Lagoa,PT,68.16,clear sky,39.0500,-27.9833,
22,Saint-Joseph,RE,71.96,few clouds,-21.3667,55.6167,
25,Nortelandia,BR,71.85,broken clouds,-14.4547,-56.8028,
28,Kapaa,US,79.48,scattered clouds,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,74.97,moderate rain,-23.1203,-134.9692,People ThankYou
9,Carnarvon,AU,68.40,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
13,Karratha,AU,79.77,clear sky,-20.7377,116.8463,ibis Styles Karratha
15,Acari,PE,68.22,broken clouds,-15.4311,-74.6158,HOSPEDAJE XIMENA
18,Truth Or Consequences,US,71.83,clear sky,33.1284,-107.2528,Charles Motel
...,...,...,...,...,...,...,...
657,Uruguaiana,BR,66.13,clear sky,-29.7547,-57.0883,Fares Turis Hotel
667,Canutama,BR,75.70,overcast clouds,-6.5339,-64.3831,Hotel Jhennifer
671,Nanakuli,US,84.15,few clouds,21.3906,-158.1547,Camp Pālehua
673,Santa Cruz,BO,69.26,few clouds,-17.8000,-63.1667,Aviador Hotel Boutique


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))